### Trajectory analysis

In [35]:
import pandas as pd
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import folium

import skmob
from skmob.preprocessing import compression, detection, clustering

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 200)
import warnings
warnings.simplefilter("ignore")

### read data, clean dat, adding auxiliary columns
 - use local time zone

In [36]:
events = pd.read_csv("GalapagostortoiseMovementEcology Programme_2009_2018.csv", sep=',', parse_dates=[2])
new_cols = pd.Series(events.columns.values).str.replace(pat = r"[-:]", repl="_")
tortoise = events.set_axis(labels = new_cols, axis =1)
tortoise.drop(tortoise.index[tortoise["manually_marked_outlier"].notna()], inplace = True)
tortoise.drop(columns =["manually_marked_outlier"], inplace=True)
tortoise.drop(index = tortoise.index[tortoise.isna().any(axis=1)], inplace=True)
cols = ['event_id', 'timestamp', 'location_long', 'location_lat',
       'eobs_temperature', 'ground_speed', 'heading', 'height_above_ellipsoid',
       #'individual_taxon_canonical_name', 'tag_local_identifier',
       'individual_local_identifier']
tortoise = tortoise[cols].assign()
tortoise.insert(loc = 2, column = "timestamp_UTC",
    value = tortoise['timestamp'].apply(lambda x: x.tz_localize(tz='UTC'))
)
tortoise.insert(loc = 3, column = "timestamp_local", 
    value = tortoise["timestamp_UTC"].apply(lambda x: x.tz_convert(tz='US/Pacific'))
)

tortoise.sort_values(by = ["individual_local_identifier", "timestamp_local"], inplace=True)

tortoise.insert(loc = 4, column = "minute_diff", 
    value = tortoise.groupby(['individual_local_identifier'])["timestamp_local"].diff()\
        .apply(lambda x: x/np.timedelta64(1, 'm')).fillna(0).astype('int64')
)

tortoise['year'] = tortoise["timestamp_local"].dt.year
tortoise['month'] = tortoise["timestamp_local"].dt.month
tortoise['date'] = tortoise["timestamp_local"].dt.date

In [187]:
from skmob.tessellation import tilers
from skmob.preprocessing import filtering

tessellation = tilers.tiler.get("squared", base_shape="isla santa cruz", meters=100)
tessellation["tile_lng"] = tessellation.geometry.centroid.x
tessellation["tile_lat"] = tessellation.geometry.centroid.y


<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 100183 entries, 0 to 100182
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype   
---  ------    --------------   -----   
 0   tile_ID   100183 non-null  object  
 1   geometry  100183 non-null  geometry
 2   tile_lng  100183 non-null  float64 
 3   tile_lat  100183 non-null  float64 
dtypes: float64(2), geometry(1), object(1)
memory usage: 3.1+ MB


In [188]:
tortoise_trajectory = skmob.TrajDataFrame(tortoise, latitude='location_lat', longitude='location_long', \
    user_id='individual_local_identifier', datetime='timestamp_local', \
)
tortoise_trajectory_mapping = tortoise_trajectory.mapping(tessellation) # map trajectory into tessellation tiles
tortoise_tiled = tortoise_trajectory_mapping.merge(tessellation, on = "tile_ID") # join to get the tile coordinates
tortoise_tiled.drop(columns =["lat","lng"], inplace=True) # drop the original lat/lng so that we can create new trajectory
tortoise_tiled.sort_values(by = ["uid", "datetime"], inplace=True) # sort before making trajectory
tortoise_tiled_trajectory = skmob.TrajDataFrame(tortoise_tiled, latitude='tile_lat', longitude='tile_lng', \
    user_id='uid', datetime='datetime', \
)
tortoise_trajectory_compressed = compression.compress(tortoise_trajectory, spatial_radius_km=0.02) # compressed trajectory
tortoise_tiled_trajectory_compressed = compression.compress(tortoise_tiled_trajectory, spatial_radius_km=0.02) # compressed trajectory

 - filter out those for tortoise 'Alison' for ease of visualization while developing code

In [213]:
alison = tortoise[tortoise["individual_local_identifier"] == "Alison"]
alison_trajectory = tortoise_trajectory.query("uid == 'Alison'")
alison_trajectory_mapping = tortoise_trajectory_mapping.query("uid == 'Alison'")
alison_tiled = tortoise_tiled.query("uid == 'Alison'")
alison_tiled_trajectory = tortoise_tiled_trajectory.query("uid == 'Alison'")
alison_trajectory_compressed = tortoise_trajectory_compressed.query("uid == 'Alison'")
alison_tiled_trajectory_compressed = tortoise_tiled_trajectory_compressed.query("uid == 'Alison'")
#alison.shape, alison_trajectory.shape, alison_trajectory_mapping.shape, alison_tiled_trajectory.shape, alison_trajectory_compressed.shape, alison_tiled_trajectory_compressed.shape

In [224]:
alison_tiled_count = alison_trajectory_mapping['tile_ID'].value_counts()

In [229]:
temp_df1 = pd.DataFrame({'tile_ID': alison_tiled_count.index, 'count': alison_tiled_count.values})
temp_df1.set_index("tile_ID",inplace=True)
tessellation_count = tessellation.set_index('tile_ID')
tessellation_count = tessellation_count.merge(temp_df1, on = "tile_ID")

In [245]:
import branca.colormap as cm

import math
linear = cm.LinearColormap(['green', 'yellow', 'red'], vmin=0, vmax=math.log(alison_tiled_count.max()))


f = folium.Figure(width=1000, height=500)
m = folium.Map(location=[-0.65, -90.35], zoom_start=12, min_zoom=2, max_bounds=True, tiles='cartodbpositron').add_to(f)
folium.GeoJson(tessellation_count, style_function=lambda feature: {
        'fill': True,
        'color': 'lightgrey' if feature['properties']['count']==0 else \
                linear(math.log(float(feature['properties']['count']))),
                #linear.to_step(n=6,data = [30.6, 50, 51, 52, 53, 54, 55, 60, 70, 100],  method = 'quantiles')(float(feature['properties']['count'])),
        'weight' : 0.3,
        'fillOpacity' : 0.5,
        'label': "abcd"
        }).add_to(m)

m

In [102]:

f = folium.Figure(width=1000, height=500)
m = folium.Map(location=[-0.65, -90.35], zoom_start=12, min_zoom=2, max_bounds=True).add_to(f)
folium.GeoJson(tessellation).add_to(m)
m